## **Turi Image Classifier**

Welcome to the image classification model quickstart on Skafos! The purpose of this notebook is to get you going end-to-end. Below we will do the following:

1. Load 25,000 cat and dog images.
2. Build a classification model to predict whether an image is a cat or a dog.
3. Convert the model to CoreML and save it to the Skafos framework.

The example below is based on [Turi Create's Image Classifier](https://apple.github.io/turicreate/docs/userguide/image_classifier/).

---

Execute each cell one-by-one by selecting the cell and doing one of the following:
- Clicking the play button at the top of this frame 
- Typing 'Control + Enter' or 'Shift + Enter'

In [ ]:
# Import necessary libraries
import urllib
import tarfile

import coremltools
from skafossdk import *

from utilities.load_turicreate import *
import utilities.save_models as sm

In [ ]:
tc = install_turicreate(timeout=500, retries=2) #check to make sure turicreate is installed
ska  = Skafos() # initialize Skafos

### 1. **Load the data**
The training data for this example are 25,000 images, 12,500 cats and 12,500 dogs. The original data set is [here](https://www.microsoft.com/en-us/download/details.aspx?id=54765) and we have also included it in the public bucket listed below. First, we need to unzip and extract the images. 

In [ ]:
# Fetch training data from an AWS bucket publicly hosted by Metis Machine
data_url = "https://s3.amazonaws.com/skafos.example.data/ImageClassifier/PetImages.tar.gz"
data_path = "PetImages.tar.gz"

# Extract images from the zipped tar file 
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

After extracting the images, they need to be loaded into Turi Create SFrame. Labels are then applied to each image (e.g. dog or cat) and the data is randomly split into train and test sets, where 80% of the data is used for training and 20% is used for model evaluation (if you desire). 

In [ ]:
# Load images (Note: you can ignore 'Unexpected JPEG decode failure' errors)
data = tc.image_analysis.load_images('PetImages', with_path=True, ignore_failure=True)

# From the path-name, create a label column. This labels each image as either a dog or a cat 
data['label'] = data['path'].apply(lambda path: 'dog' if '/Dog' in path else 'cat')

# Make a train-set split
train_data, test_data = data.random_split(0.8)

### 2. **Build the model**
We use the `tc.image_classifer.create` function and specify the data, target variable, and a few other arguments needed to properly train the model. To understand more about this specific function, check out the [Ture Create Documentation](https://apple.github.io/turicreate/docs/api/generated/turicreate.image_classifier.create.html#turicreate.image_classifier.create).

**Note: The code below has been commented because we highly recommend running this model using a GPU.  If you uncomment the second line and run this cell as written, it will use a CPU, take approximately 90 minutes, and will re-create the cat/dog predictor model. Once you have loaded your own data, uncomment these lines to train your image classification model. It will currently use a CPU, but GPU support is coming soon.**

In [ ]:
# Train an image classification model, specifying the name of the 'target'/'label' column
#model = tc.image_classifier.create(train_data, target='label')

### 3. **Save the model**
Once your model has been created, it must be saved to the Skafos framework via the code below. This will trigger a push to your mobile app. Note that you will need to uncomment the lines below to do this. 

In [ ]:
## Export the model to CoreML
#coreml_model_name = 'ImageClassifier.mlmodel'
#res = model.export_coreml(coreml_model_name)

## Use coremltools to convert model weights to half-precision.
## This may be necessary if you have memory concerns within your app
#model_spec = coremltools.utils.load_spec(coreml_model_name)
#model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
#coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# Compress the model
#compressed_model = sm.compress_model(coreml_model_name)

# Save to Skafos
#sm.skafos_save_model(
#    skafos=ska, 
#    model_name=coreml_model_name,
#    compressed_model=compressed_model,
#    permissions='public'
# )